In [13]:
import gym

In [14]:
!pip install tensorflow==2.3.1

In [ ]:
!pip install gym gym[atari]

In [ ]:
!pip install keras-rl2

In [ ]:
!pip install pyvirtualdisplay

In [ ]:
!sudo apt-get install xvfb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,270 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.9 [784 kB]
Fetched 784 kB in 0s (7,498 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

In [15]:
from pyvirtualdisplay import Display
_display = Display(visible=False, size=(1400, 900))
_ = _display.start()

In [16]:
env= gym.make('SpaceInvaders-v0')

In [17]:
env.action_space #has 6 discrete actions

Discrete(6)

In [18]:
episodes=10

for episode in range(1,episodes):
    state=env.reset() #resetting env at start
    done=False #done flag to tell agent has completed
    score=0
    
    while not done:
        env.render()
        state,reward,done,info=env.step(env.action_space.sample())#does random action in action space
        score+=reward
    print('Episode: {}\nScore: {}'.format(episode,score))
env.close()

Episode: 1
Score: 60.0
Episode: 2
Score: 140.0
Episode: 3
Score: 155.0
Episode: 4
Score: 135.0
Episode: 5
Score: 380.0
Episode: 6
Score: 135.0
Episode: 7
Score: 110.0
Episode: 8
Score: 180.0
Episode: 9
Score: 105.0


In [19]:
#imports
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam

In [20]:
def build_model(height, width, channels, actions):
    model= Sequential()#sequential layer
    model.add(Conv2D(32,(8,8),strides=(4,4), activation='relu', input_shape=(3, height, width,channels)))#32 no of filters 8 is filter size and 4 is stride width
    model.add(Conv2D(64,(4,4), strides=(2,2), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model

In [21]:
height, width,channels=env.observation_space.shape
actions=env.action_space.n

In [29]:
model=build_model(height,width,channels,actions)

In [30]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [31]:
def build_agent(model,actions):
    policy=LinearAnnealedPolicy(EpsGreedyQPolicy(),attr='eps',value_max=1, value_min=.1,value_test=.2, nb_steps=10000)
    memory=SequentialMemory(limit=2000,window_length=3)
    dqn=DQNAgent(model=model, memory=memory,policy=policy,enable_dueling_network=True,dueling_type='avg',nb_actions=actions, nb_steps_warmup=1000)
    return dqn

In [32]:
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

In [28]:
#del model 
#uncomment and run if you get memory error in below cell


In [33]:
dqn=build_agent(model,actions)

In [34]:
dqn.compile(Adam(lr=0.0001))

In [ ]:
'''
Possible optimization:
* add more conv layers
* add more dense layers
* increase dataset size by increasing sequential memory limit(currently 2000)
* Change greedy max and min
* more training
* change learning rate
'''

In [ ]:
dqn.fit(env,nb_steps=40000,visualize=False,verbose=1)#visualization decreases training speed

Training for 40000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
10000/10000 [==============================] - 588s 59ms/step - reward: 0.2195
14 episodes - episode_reward: 155.357 [25.000, 715.000] - loss: 2.449 - mean_q: 10.955 - mean_eps: 0.505 - ale.lives: 2.217

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 622s 62ms/step - reward: 0.3110
13 episodes - episode_reward: 203.846 [50.000, 490.000] - loss: 1.547 - mean_q: 10.995 - mean_eps: 0.100 - ale.lives: 2.043

Interval 3 (20000 steps performed)
 8664/10000 [========================>.....] - ETA: 1:21 - reward: 0.3018

In [ ]:
scores=dqn.test(env,nb_episodes=10,visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...


NameError: ignored

In [ ]:
dqn.save_weights('models/dqn.h5f')

[WARNING] models/dqn.h5f.index already exists - overwrite? [y/n]n


In [ ]:
dqn.load_weights('models/dqn.h5f')